In [1]:
# !pip -q install torch_xla[tpu] -f https://storage.googleapis.com/tpu-pytorch/wheels/colab.html
# !pip -q install optuna
# !pip -q install gradio
!pip -q install wandb
!pip -q install datasets
!pip -q install evaluate
!pip -q install huggingface_hub
!pip -q install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
i

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import librosa
import evaluate
import wandb
# import gradio as gr
from datasets import load_from_disk

from datasets import Dataset
from IPython.display import Audio
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer, BitsAndBytesConfig, EarlyStoppingCallback

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!wandb login

wandb: Currently logged in as: firdhoworking (firdhoworking-sepuluh-nopember-institute-of-technology). Use `wandb login --relogin` to force relogin


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path =  '/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/df_combined.csv'

df_voice = pd.read_csv(path)

df_voice.head()

,Path,Emotion
0,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral
1,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral
2,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral
3,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral
4,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,calm


In [6]:
df_voice['Emotion'].value_counts()

,count
Emotion,
sad,752
happy,752
angry,752
neutral,716
disgust,652
fearful,652
surprised,652
calm,192


In [7]:
df_voice = df_voice[df_voice['Emotion'] != 'calm']

In [8]:
tags = np.unique(df_voice['Emotion']) # get unique category
num_tags = len(tags) # get the number of category, here we have 2 tags/categories
label2id = {t: i for i, t in enumerate(tags)} # make a dictionary to map label to id
id2label = {i: t for i, t in enumerate(tags)} # make a dictionary to map id to label

In [9]:
label2id, id2label

({'angry': 0,
  'disgust': 1,
  'fearful': 2,
  'happy': 3,
  'neutral': 4,
  'sad': 5,
  'surprised': 6},
 {0: 'angry',
  1: 'disgust',
  2: 'fearful',
  3: 'happy',
  4: 'neutral',
  5: 'sad',
  6: 'surprised'})

In [10]:
train_data = df_voice.sample(frac=0.8, random_state=42)

train_data.head()

,Path,Emotion
1857,/content/drive/MyDrive/Audio/Speech_TESS/OAF_h...,neutral
3784,/content/drive/MyDrive/Audio/Speech_TESS/YAF_r...,sad
2918,/content/drive/MyDrive/Audio/Speech_TESS/YAF_b...,disgust
2572,/content/drive/MyDrive/Audio/Speech_TESS/OAF_s...,surprised
780,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral


In [11]:
test_data = df_voice.drop(train_data.index)

test_data.head()

,Path,Emotion
3,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,neutral
12,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,happy
13,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,happy
17,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,happy
18,/content/drive/MyDrive/Audio/Speech_RAVDESS/Ac...,happy


In [ ]:
X_train = Dataset.from_pandas(train_data)
X_test = Dataset.from_pandas(test_data)

In [ ]:
import numpy as np
import librosa
from datasets import Dataset

# Fungsi untuk membaca audio dan mengubahnya menjadi array menggunakan librosa
def read_audio(path):
    # Membaca audio menggunakan librosa
    array, sampling_rate = librosa.load(path, sr=None)
    return array, sampling_rate

# Fungsi untuk mengonversi data dalam dataset menjadi format yang diinginkan
def convert_example(example):
    audio_path = example['Path']
    array, sampling_rate = read_audio(audio_path)
    return {
        'audio': {
            'path': audio_path,
            'array': array,
            'sampling_rate': sampling_rate
        },
        'labels': label2id[example['Emotion']]
    }

# Fungsi untuk mengonversi seluruh dataset
def convert_dataset(dataset):
    converted_examples = []
    for example in dataset:
        converted_example = convert_example(example)
        converted_examples.append(converted_example)
    return Dataset.from_dict(converted_examples)

converted_train = X_train.map(convert_example)
converted_test = X_test.map(convert_example)

Map:   0%|          | 0/3942 [00:00<?, ? examples/s]

Map:   0%|          | 0/986 [00:00<?, ? examples/s]

In [ ]:
converted_train, converted_test

(Dataset({
     features: ['Path', 'Emotion', '__index_level_0__', 'audio', 'labels'],
     num_rows: 3942
 }),
 Dataset({
     features: ['Path', 'Emotion', '__index_level_0__', 'audio', 'labels'],
     num_rows: 986
 }))

In [ ]:
converted_train = converted_train.remove_columns(["Path", "Emotion", "__index_level_0__"])
converted_test = converted_test.remove_columns(["Path", "Emotion", "__index_level_0__"])

In [ ]:
converted_train, converted_test

(Dataset({
     features: ['audio', 'labels'],
     num_rows: 3942
 }),
 Dataset({
     features: ['audio', 'labels'],
     num_rows: 986
 }))

In [ ]:
converted_train.save_to_disk('/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/converted_train')
converted_test.save_to_disk('/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/converted_test')

Saving the dataset (0/4 shards):   0%|          | 0/3942 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/986 [00:00<?, ? examples/s]

In [12]:
converted_train = load_from_disk('/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/converted_train')
converted_test = load_from_disk('/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/converted_test')

In [13]:
# model_id = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
model_id = "openai/whisper-large-v3"

feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [15]:
sample = converted_train[0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: -3.36e-05, Variance: 0.000273


In [16]:
max_duration = 30.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [17]:
data_encoded_train = converted_train.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

data_encoded_train

Dataset({
    features: ['labels', 'input_features'],
    num_rows: 3942
})

In [18]:
data_encoded_test = converted_test.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

data_encoded_test

Dataset({
    features: ['labels', 'input_features'],
    num_rows: 986
})

In [19]:
num_labels = len(id2label)

In [ ]:
# def model_init():
#     return AutoModelForAudioClassification.from_pretrained(
#         model_id,
#         num_labels=num_labels,
#         label2id=label2id,
#         id2label=id2label,
#     )

In [ ]:
# def hp_space_optuna(trial):
#     return {
#         "learning_rate": trial.suggest_categorical("learning_rate", [1e-2, 1e-3, 1e-4, 5e-2, 5e-3, 5e-4]),
#         "gradient_accumulation_steps": trial.suggest_int("gradient_accumulation_steps", 1, 5),
#         "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
#         "per_device_eval_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
#         "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.3),
#         "weight_decay": trial.suggest_float("weight_decay", 0.00, 0.3),
#         "logging_steps": trial.suggest_int("logging_steps", 5, 50, step=5),
#         "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "constant", "constant_with_warmup"])  # Tune lr scheduler type
#     }

In [ ]:
# training_args = TrainingArguments(
#     # f"speech-emotion-recognition-with-wav2vec2-xlsr",
#     output_dir="/content/drive/MyDrive/SPEECH EMOTION RECOGNITION/TUNING",
#     num_train_epochs=3,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     fp16=True,
#     push_to_hub=False
# )

In [ ]:
# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     labels = eval_pred.label_ids

#     accuracy = accuracy_score(labels, predictions)

#     precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }

In [ ]:
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=data_encoded_train,
#     eval_dataset=data_encoded_test,
#     tokenizer=feature_extractor,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# best_run = trainer.hyperparameter_search(
#     direction="maximize",
#     hp_space=hp_space_optuna,
#     n_trials=10,
#     backend="optuna",
#     compute_objective=lambda metrics: metrics["accuracy"],
#     study_name="speech-emotion-recognition-with-wav2vec2-xlsr",
# )

In [ ]:
# print(f"Best hyperparameters: {best_run.hyperparameters}")

In [ ]:
# best_hyperparameters = best_run.hyperparameters

In [ ]:
# training_args = TrainingArguments(
#     f"speech-emotion-recognition-with-wav2vec2-xlsr",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=best_hyperparameters['learning_rate'],
#     gradient_accumulation_steps=best_hyperparameters['gradient_accumulation_steps'],
#     per_device_train_batch_size=best_hyperparameters['per_device_train_batch_size'],
#     num_train_epochs=best_hyperparameters['num_train_epochs'],
#     warmup_ratio=best_hyperparameters['warmup_ratio'],
#     logging_steps=best_hyperparameters['logging_steps'],
#     lr_scheduler_type=best_hyperparameters['lr_scheduler_type'],
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     load_best_model_at_end=True,
#     push_to_hub=True,
#     fp16=True
# )

In [ ]:
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=data_encoded_train,
#     eval_dataset=data_encoded_test,
#     tokenizer=feature_extractor,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# trainer.train()

In [ ]:
# evaluation_results = trainer.evaluate()
# print(evaluation_results)

In [20]:
model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-large-v3 and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
wandb.init(project="speech-emotion-recognition-with-whisper")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: firdhoworking (firdhoworking-sepuluh-nopember-institute-of-technology). Use `wandb login --relogin` to force relogin


In [22]:
batch_size = 2  # Updated batch size
gradient_accumulation_steps = 5  # Updated gradient accumulation steps
num_train_epochs = 25  # Updated number of epochs

training_args = TrainingArguments(
    f"speech-emotion-recognition-with-openai-whisper-large-v3",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    lr_scheduler_type="linear",
    report_to="wandb",
    push_to_hub=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08)
lr_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, end_factor=1.0, total_iters=len(train_data) * training_args.num_train_epochs)

In [24]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [25]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=data_encoded_train,
    eval_dataset=data_encoded_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [26]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.494800,0.491117,0.828600,0.844875,0.828600,0.830209
1,0.627100,0.530747,0.822515,0.855950,0.822515,0.827671
2,0.236400,0.507580,0.869168,0.872654,0.869168,0.868435
3,0.015600,0.566859,0.873225,0.886819,0.873225,0.874537
5,0.011200,0.470067,0.910751,0.915914,0.910751,0.911420
6,0.001300,0.523163,0.913793,0.920424,0.913793,0.913685
7,0.189400,0.500845,0.919878,0.922969,0.919878,0.919849
8,0.087700,0.551722,0.913793,0.915193,0.913793,0.913813
10,0.002600,0.833375,0.877282,0.894921,0.877282,0.876995


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead.

TrainOutput(global_step=4336, training_loss=0.21827457035219747, metrics={'train_runtime': 21170.2062, 'train_samples_per_second': 4.655, 'train_steps_per_second': 0.465, 'total_flos': 6.347817440084885e+19, 'train_loss': 0.21827457035219747, 'epoch': 10.99949264332826})

In [27]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.5008450746536255, 'eval_accuracy': 0.9198782961460447, 'eval_precision': 0.9229688914731574, 'eval_recall': 0.9198782961460447, 'eval_f1': 0.9198486992428615, 'eval_runtime': 250.7265, 'eval_samples_per_second': 3.933, 'eval_steps_per_second': 1.966, 'epoch': 10.99949264332826}


In [28]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/firdhokk/speech-emotion-recognition-with-openai-whisper-large-v3/commit/ede9acef7d7ddecac806bce677640371b95cb186', commit_message='End of training', commit_description='', oid='ede9acef7d7ddecac806bce677640371b95cb186', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
wandb.finish()

eval/accuracy,▁▁▄▅▇▇███▆▅█
eval/f1,▁▁▄▅▇▇███▆▅█
eval/loss,▂▂▂▃▁▁▂▂▃▃█▂
eval/precision,▁▂▃▅▇▇██▇▆▅█
eval/recall,▁▁▄▅▇▇███▆▅█
eval/runtime,▁▃▄▄▅▅▅▅▅▆█▆
eval/samples_per_second,█▆▅▅▄▄▄▄▄▃▁▃
eval/steps_per_second,█▆▅▅▄▄▄▄▄▃▁▃
train/epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/grad_norm,▃▂▂▂▂▁▃▃▄▁▅▃▁▇▁▃▁▁▁▁█▁▁▁▃▁▁▅▁▁▁▁▁▁▁▁▁▃▁▁
